In [59]:
import vrep 
import sys
import time 
import numpy as np
import skfuzzy as fuzz

from tank import *

In [60]:
vrep.simxFinish(-1) # closes all opened connections, in case any prevoius wasnt finished
clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # start a connection

if clientID!=-1:
    print ("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

#create instance of Tank
tank=Tank(clientID)

Connected to remote API server


In [61]:
proximity_sensor="Proximity_sensor"


# get handle to proximity sensor
err_code,proximity_sensors_handle = vrep.simxGetObjectHandle(clientID,"Proximity_sensor", vrep.simx_opmode_blocking)
    
#read and print values from proximity sensors
#first reading should be done with simx_opmode_streaming, further with simx_opmode_buffer parameter

err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,proximity_sensors_handle,vrep.simx_opmode_streaming)

In [62]:
tank.rightvelocity

0

In [63]:
# proximity sensor value starts in this scene from above 7 and might be as low as 0 when buliding is hit
# we will define current speed of tank and distance from building as inputs to control systems and change in velocity as an output

In [64]:
in_dist = np.arange(0.5,7.2,0.1)
in_velo = np.arange(0,10,0.1)
out_velo = np.arange(0,10,0.1)

# Generate fuzzy membership functions
in_dist_lo = fuzz.trimf(in_dist,[0.5,0.5,3.5])
in_dist_mid = fuzz.trimf(in_dist,[0.5,3.5,5.5])
in_dist_high = fuzz.trimf(in_dist,[3.5,7.2,7.2])

# in speed 

in_velo_lo = fuzz.trimf(in_velo,[0,0,5])
in_velo_mid = fuzz.trimf(in_velo,[0,5,10])
in_velo_high = fuzz.trimf(in_velo,[5,10,10])

out_velo_lo = fuzz.trimf(out_velo,[0,0,5])
out_velo_mid = fuzz.trimf(out_velo,[0,5,10])
out_velo_high = fuzz.trimf(out_velo,[5,10,10])

In [65]:
def velo_update(dist,velo):

    # Generate fuzzy membership functions
    dist_level_lo = fuzz.interp_membership(in_dist,in_dist_lo,dist)
    dist_level_mid = fuzz.interp_membership(in_dist,in_dist_mid,dist)
    dist_level_high = fuzz.interp_membership(in_dist,in_dist_high,dist)

    # in speed 

    in_velo_level_lo = fuzz.interp_membership(in_velo,in_velo_lo,velo)
    in_velo_level_mid = fuzz.interp_membership(in_velo,in_velo_mid,velo)
    in_velo_elvel_high = fuzz.interp_membership(in_velo,in_velo_high,velo)




    # now rules
    
    # decreasing velocity rules
    # we need to decrease speed if distance is low (with any speed) or if distance is mid and velocity is high 
    
    dec_rule1 = np.fmin(dist_level_lo,in_velo_level_lo)
    dec_rule2 = np.fmin(dist_level_mid,in_velo_elvel_high)
    
    # or operator = max and operatior = min
    
    dec_rule = np.fmax(dec_rule1,dec_rule2)
    
    # now some rules for not doing anything 
    
    no_rule1 = np.fmin(dist_level_mid,in_velo_level_mid)
    no_rule2 = np.fmin(dist_level_high,in_velo_elvel_high)
    
    no_rule = np.fmin(no_rule1,no_rule2)
    
    
    # acceleration
    
    acc_rule1 =np.fmin(dist_level_high,in_velo_level_lo)
    acc_rule2 =np.fmin(dist_level_high,in_velo_level_mid)
    acc_rule3 = np.fmin(dist_level_mid,in_velo_elvel_high)
    
    acc_rule = np.fmax(acc_rule1,np.fmax(acc_rule2,acc_rule3))
    
    
    # acitvation to get output velocity 
    dec_activation = np.fmin(dec_rule,out_velo_lo)
    no_activation = np.fmin(no_rule,out_velo_mid)
    acc_activation = np.fmin(acc_rule,out_velo_high)
    # Aggregate all three output membership functions together
    aggregated = np.fmax(dec_activation,np.fmax(no_activation,acc_activation))
    
    # Calculate defuzzified result
    new_velo = fuzz.defuzz(out_velo, aggregated, 'centroid')    
    
    
    return new_velo



In [66]:

tank.forward(5)
#continue reading and printing values from proximity sensors
# right and left velocity seems to be the same so will be using right velocity if needed to read it 
# tank.rightvelocity()
velo = 0
while True: # read values for 5 seconds
    
    err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,proximity_sensors_handle,vrep.simx_opmode_buffer )
    
    if (err_code == 0):
        distance_data = np.linalg.norm(detectedPoint)
        print("Proximity_sensor",distance_data,"Velocity",tank.rightvelocity)
      
        # controler 
        velo = velo_update(distance_data,tank.rightvelocity)
        tank.forward(velo)
      
    

Proximity_sensor 7.1738242444239795 Velocity 5


EmptyMembershipError: The membership function area is empty.